# Zellenradschleuse Read/Write Cycle Example

This notebook demonstrates:

1. Discovering Metexon devices (BLE) filtered by the Metexon primary service.
2. Connecting with the typed `ZellenradschleuseClient`.
3. Reading the current system state.
4. Performing a partial manual control update using sentinel-aware helper utilities.
5. Updating the RGB LEDs.
6. Re-reading system state.

> Requires real hardware in range advertising the Metexon service.

If no device is found the notebook will stop early.

In [ ]:
# Discovery step
import sys
sys.coinit_flags = 0  # 0 means MTA

from metexon import adiscover_metexon
import platform
if platform.system() == 'Windows':
    try:
        from bleak.backends.winrt.util import allow_sta
        # tell Bleak we are using a graphical user interface that has been properly
        # configured to work with asyncio
        allow_sta()
    except ImportError:
        # other OSes and older versions of Bleak will raise ImportError which we
        # can safely ignore
        pass

# In Jupyter the event loop is already running, so use the async discover function
FOUND = await adiscover_metexon(timeout=5.0)
if not FOUND:
    raise SystemExit("No Metexon devices discovered; cannot continue example.")
FOUND[0]

AttributeError: module 'ctypes' has no attribute 'WINFUNCTYPE'

In [ ]:
# Connect and read
from metexon.zellenradschleuse import ZellenradschleuseClient

address = FOUND[0]['address']
client = ZellenradschleuseClient(address)
client.connect()
try:
    print("Device type:", client.device_type())
    ss = client.read_system_state()
    print("Initial state:", ss.to_json())
finally:
    pass  # keep connection open for next cell

In [ ]:
# Change LEDs
client.write_rgb_led([(255,127,63), (0,255,0)])
print("Updated LEDs")

In [ ]:
# Re-read system state and disconnect
updated = client.read_system_state()
print("Updated state:", updated.to_json())
client.disconnect()
print("Disconnected")